In [1]:
from deepdist import DeepDist
from gensim.models.word2vec import Word2Vec
from pyspark import SparkContext

sc = SparkContext()

In [22]:
corpus = sc.textFile('hdfs:///user/hadoop/data/train-unsup.txt').map(lambda s: s.split())

In [3]:
def gradient(model, sentences):  # executes on workers
    syn0, syn1 = model.wv.syn0.copy(), model.syn1neg.copy()
    model.train(sentences,total_examples=model.corpus_count,epochs=1)
    print syn0,model.wv.syn0
    return {'syn0': model.wv.syn0 - syn0, 'syn1': model.syn1neg - syn1}

def descent(model, update):      # executes on master
    model.wv.syn0 += update['syn0']
    model.syn1neg += update['syn1']
    print model.syn1neg

In [5]:
%%time
l = corpus.collect()
with DeepDist(Word2Vec(l),master='192.168.91.1:5000') as dd:
    for i in range(1):
        dd.train(corpus,gradient,descent)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)



*** Master: 192.168.91.1:5000

Listening to 0.0.0.0:5000...


192.168.91.3 - - [20/Apr/2018 23:18:29] "GET /model HTTP/1.1" 200 -
192.168.91.2 - - [20/Apr/2018 23:18:29] "GET /model HTTP/1.1" 200 -
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `syn1neg` (Attribute will be removed in 4.0.0, use self.trainables.syn1neg instead).
  # Remove the CWD from sys.path while we load stuff.
192.168.91.2 - - [20/Apr/2018 23:18:37] "POST /update HTTP/1.1" 200 -
192.168.91.3 - - [20/Apr/2018 23:18:37] "POST /update HTTP/1.1" 200 -
192.168.91.1 - - [20/Apr/2018 23:18:37] "POST /shutdown HTTP/1.1" 200 -


[[ 0.0608131  -0.12580377  0.06759548 ... -0.09786701 -0.1861918
   0.36205325]
 [ 0.09102792 -0.38518488  0.3282643  ... -0.42333406 -0.33153453
   0.3457613 ]
 [ 0.00151996 -0.13128579  0.23031294 ... -0.395289   -0.16744219
   0.518365  ]
 ...
 [-0.02124884 -0.21832116  0.1927367  ... -0.06189404 -0.13340291
   0.13252759]
 [-0.03612383 -0.28568524  0.19085966 ... -0.09000628 -0.11676696
   0.2034887 ]
 [-0.06165538 -0.1868179   0.14940128 ... -0.1834356  -0.13112573
   0.1639142 ]]
[[ 0.123081   -0.22396344 -0.1078468  ... -0.00703365 -0.15446497
   0.23406479]
 [ 0.02560237 -0.5102339   0.23763654 ... -0.26789477 -0.22810525
   0.41631782]
 [-0.0294379  -0.06242377  0.19947916 ... -0.21051572 -0.06683789
   0.40247545]
 ...
 [-0.04267003 -0.17610325  0.2019635  ... -0.02938834 -0.13805445
   0.13195014]
 [-0.05665994 -0.29897386  0.21977861 ... -0.05119795 -0.10687561
   0.19771262]
 [-0.07567409 -0.15767807  0.20743802 ... -0.17878541 -0.15254727
   0.13116768]]
Exit requested...

In [8]:
corpus =sc.textFile('hdfs:///user/hadoop/data/test-neg.txt').map(
    lambda s: s.split())

In [9]:
list1 = corpus.collect()
tags = []
tag=''
for i in range(1,12501): 
    tag='tag'+ str(i)
    tags.append(tag)
    tag='' 

i=0
result = []
for l in list1: 
    T=TaggedDocument(l,tags[i])
    i=i+1 
    result.append(T)
    
print result[:2]


[TaggedDocument(words=[u'once', u'again', u'mr', u'costner', u'has', u'dragged', u'out', u'a', u'movie', u'for', u'far', u'longer', u'than', u'necessary', u'aside', u'from', u'the', u'terrific', u'sea', u'rescue', u'sequences', u'of', u'which', u'there', u'are', u'very', u'few', u'i', u'just', u'did', u'not', u'care', u'about', u'any', u'of', u'the', u'characters', u'most', u'of', u'us', u'have', u'ghosts', u'in', u'the', u'closet', u'and', u'costners', u'character', u'are', u'realized', u'early', u'on', u'and', u'then', u'forgotten', u'until', u'much', u'later', u'by', u'which', u'time', u'i', u'did', u'not', u'care', u'the', u'character', u'we', u'should', u'really', u'care', u'about', u'is', u'a', u'very', u'cocky', u'overconfident', u'ashton', u'kutcher', u'the', u'problem', u'is', u'he', u'comes', u'off', u'as', u'kid', u'who', u'thinks', u'hes', u'better', u'than', u'anyone', u'else', u'around', u'him', u'and', u'shows', u'no', u'signs', u'of', u'a', u'cluttered', u'closet', u'hi

In [10]:
def gradient(model, taggedData):  # executes on workers
    syn0, syn1, doctag = model.wv.syn0.copy(), model.syn1neg.copy(), model.docvecs.doctag_syn0.copy()
    print syn0,syn1,taggedData[1]
    model.train(taggedData,total_examples=model.corpus_count,epochs=1)
    return {'syn0': model.wv.syn0 - syn0, 'syn1': model.syn1neg - syn1, 'doctag': model.docvecs.doctag_syn0-doctag }

def descent(model, update):      # executes on master
    model.wv.syn0 += update['syn0']
    model.syn1neg += update['syn1']
    model.docvecs.doctag_syn0 += update['doctag']
    print model.syn1neg

In [11]:
%%time
with DeepDist(Doc2Vec(result),master='192.168.91.1:5000') as dd:
    dd.train(corpus,gradient,descent)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)



*** Master: 192.168.91.1:5000

Listening to 0.0.0.0:5000...


192.168.91.3 - - [20/Apr/2018 23:22:18] "GET /model HTTP/1.1" 200 -
192.168.91.2 - - [20/Apr/2018 23:22:18] "GET /model HTTP/1.1" 200 -
192.168.91.1 - - [20/Apr/2018 23:26:46] "POST /shutdown HTTP/1.1" 200 -


Exit requested...


KeyboardInterrupt: 

In [20]:
corpus =sc.textFile('hdfs:///user/hadoop/data/train-neglabeld').map(
    lambda s: TaggedDocument(s[19:].split(), s[:18]))

In [21]:
list = corpus.collect()

KeyboardInterrupt: 